In [4]:
import io
import zipfile
import requests
import pandas as pd

In [ ]:
BASE_URL = "https://transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip"

def load_month(year, month):
    url = BASE_URL.format(year=year, month=month)
    print(f"Fetching {url} ...")
    r = requests.get(url)
    if r.status_code != 200:
        print(f"  Skipping {year}-{month:02d} (HTTP {r.status_code})")
        return None

    # Treat response as zip in memory
    zf = zipfile.ZipFile(io.BytesIO(r.content))
    # Assume first CSV in zip is the data
    csv_name = [n for n in zf.namelist() if n.lower().endswith(".csv")][0]
    with zf.open(csv_name) as f:
        df = pd.read_csv(f, low_memory=False)
    print(f"  Loaded {len(df):,} rows for {year}-{month:02d}")
    return df


all_dfs = []

# 2018–2024: all 12 months
for year in range(2018, 2025):
    for month in range(1, 13):
        df = load_month(year, month)
        if df is not None:
            all_dfs.append(df)

# 2025: only January (adjust if more months become available)
df_2025_01 = load_month(2025, 1)
if df_2025_01 is not None:
    all_dfs.append(df_2025_01)

# Combine into one big DataFrame
flights = pd.concat(all_dfs, ignore_index=True)
print("Final shape:", flights.shape)


In [ ]:
import pandas as pd

chunks = pd.read_csv(
    "full_flights_data.csv",
    chunksize=500_000,      # 500k rows per chunk
)

sampled_chunks = []

for chunk in chunks:
    # take 5% of each chunk (tune this)
    sampled = chunk.sample(frac=0.05, random_state=42)
    sampled_chunks.append(sampled)

flights_sample = pd.concat(sampled_chunks, ignore_index=True)
print(flights_sample.shape)


In [128]:
#Dropping ISN - Permenantly closed airport
mask_isn = (flights_sample["Origin"] == "ISN") | (flights_sample["Dest"] == "ISN")
flights_sample = flights_sample.loc[~mask_isn].reset_index(drop=True)


#Removing Cancelled and Diverted Flights, and dropping NA Delay times
normal_flights = flights_sample[(flights_sample['Cancelled'] == 0) & (flights_sample['Diverted'] == 0)]
cancelled_flights = flights_sample[flights_sample['Cancelled'] == 1]
diverted_flights = flights_sample[flights_sample['Diverted'] == 1]
print(len(normal_flights))
normal_flights = normal_flights.dropna(subset=['ArrDelay'])
normal_flights = normal_flights.drop(columns=["Unnamed: 0"])
print(len(normal_flights))

2268979
2268960


In [129]:
normal_flights[['FlightDate', 'ArrDelay', 'Origin', 'Dest']]

,FlightDate,ArrDelay,Origin,Dest
0,2018-01-26,2.0,ATL,FWA
1,2018-01-20,-16.0,MCO,BWI
2,2018-01-27,-16.0,IAH,LIT
3,2018-01-14,-13.0,IAH,JAN
4,2018-01-10,-26.0,FAT,SLC
...,...,...,...,...
2325188,2025-01-30,-16.0,LAX,LAS
2325189,2025-01-27,-2.0,FLL,IAH
2325190,2025-01-30,-18.0,EWR,LAX
2325191,2025-01-04,26.0,IAH,LGA


In [130]:
airports_set = set(normal_flights["Origin"]).union(set(normal_flights["Dest"]))
airports = pd.DataFrame({'IATA': list(airports_set)})

# Load airport reference file that you’ve downloaded
airports_ref = pd.read_csv("airports.csv")  # rename to whatever you saved

# Keep only entries with IATA codes
airports_ref = airports_ref[airports_ref['iata_code'].notna()]

airports_ref_small = airports_ref[['iata_code', 'latitude_deg', 'longitude_deg']].rename(
    columns={
        'iata_code': 'IATA',
        'latitude_deg': 'LAT',
        'longitude_deg': 'LON'
    }
)

airports = airports.merge(
    airports_ref_small,
    on='IATA',
    how='left'
)



In [131]:
stations = pd.read_csv("isd-history.csv")

# Keep stations with coordinates
stations = stations[stations['LAT'].notna() & stations['LON'].notna()]
stations = stations[
    (stations['LAT'] != 0) &
    (stations['LON'] != 0)
]


# Build STATION ID: USAF(6 digits) + WBAN(5 digits)
stations['USAF'] = stations['USAF'].astype(str).str.zfill(6)
stations['WBAN'] = stations['WBAN'].astype(str).str.zfill(5)
stations['STATION'] = stations['USAF'] + stations['WBAN']

stations_small = stations[['STATION', 'LAT', 'LON']]

stations[stations['USAF'] == '722235'] 

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,STATION
18259,722235,13838,MOBILE DOWNTOWN AIRPORT,US,AL,KBFM,30.615,-88.063,4.9,19420701,20250827,72223513838
18260,722235,99999,MOBILE DOWNTOWN,US,AL,KBFM,30.617,-88.067,8.0,20000101,20031231,72223599999


In [135]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    """
    lat1, lon1: arrays (airports)
    lat2, lon2: arrays (stations)
    Returns distance matrix [n_airports x n_stations] in km.
    """
    R = 6371.0  # Earth radius in km
    lat1 = np.radians(lat1)[:, None]  # shape (n_airports, 1)
    lon1 = np.radians(lon1)[:, None]
    lat2 = np.radians(lat2)[None, :]  # shape (1, n_stations)
    lon2 = np.radians(lon2)[None, :]

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Drop airports with missing coords before distance calc
airports_valid = airports.dropna(subset=['LAT', 'LON']).reset_index(drop=True)

dist_matrix = haversine(
    airports_valid['LAT'].values,
    airports_valid['LON'].values,
    stations_small['LAT'].values,
    stations_small['LON'].values
)

station_ids = stations_small['STATION'].astype(str)

# USAF dummy
dummy_usaf = station_ids.str.startswith('999999')

# NEW: WBAN dummy → ends with 99999
dummy_wban = station_ids.str.endswith('99999')

# Combine
dummy_mask = dummy_usaf | dummy_wban

penalty_km = 50
dist_matrix[:, dummy_mask] += penalty_km


# For each airport, index of nearest station
nearest_idx = dist_matrix.argmin(axis=1)
nearest_dist_km = dist_matrix[np.arange(len(airports_valid)), nearest_idx]

airports_valid['nearest_station'] = stations_small.iloc[nearest_idx]['STATION'].values
airports_valid['nearest_station_dist_km'] = nearest_dist_km

airports_with_station = airports.merge(
    airports_valid[['IATA', 'nearest_station', 'nearest_station_dist_km']],
    on='IATA',
    how='left'
)

airports_with_station[airports_with_station['IATA'] == 'BFM']

,IATA,LAT,LON,nearest_station,nearest_station_dist_km
171,BFM,30.626801,-88.0681,72223513838,1.399972


In [136]:
# Use the mapping we just created
airport_map = airports_with_station[['IATA', 'nearest_station']]

# Merge origin station
normal_flights = normal_flights.merge(
    airport_map.rename(columns={'IATA': 'Origin', 'nearest_station': 'ORIGIN_STATION'}),
    on='Origin',
    how='left'
)

# Merge DESTINATION station
normal_flights = normal_flights.merge(
    airport_map.rename(columns={'IATA': 'Dest', 'nearest_station': 'DEST_STATION'}),
    on='Dest',
    how='left'
)

normal_flights[['Origin', 'ORIGIN_STATION', 'Dest', 'DEST_STATION']]


,Origin,ORIGIN_STATION,Dest,DEST_STATION
0,ATL,72219013874,FWA,72533014827
1,MCO,72205012815,BWI,72406093721
2,IAH,72243012960,LIT,72340313963
3,IAH,72243012960,JAN,72235003940
4,FAT,72389093193,SLC,72572024127
...,...,...,...,...
2268955,LAX,72295023174,LAS,72386023169
2268956,FLL,72202512849,IAH,72243012960
2268957,EWR,72502014734,LAX,72295023174
2268958,IAH,72243012960,LGA,72503014732


In [137]:
stations_needed = set(normal_flights["ORIGIN_STATION"].dropna()).union(
    set(normal_flights["DEST_STATION"].dropna())
)

stations_needed = sorted(stations_needed)
len(stations_needed)


382

In [138]:
import os
import requests

years = range(2018, 2026)   # adjust your years

base_url = "https://www.ncei.noaa.gov/data/global-hourly/access"

outdir = "noaa_data"
os.makedirs(outdir, exist_ok=True)

for year in years:
    for station in stations_needed:
        url = f"{base_url}/{year}/{station}.csv"
        outfile = f"{outdir}/{station}_{year}.csv"

        if os.path.exists(outfile):
            print("Already downloaded:", outfile)
            continue

        print("Downloading:", url)
        r = requests.get(url, stream=True)

        if r.status_code == 200:
            with open(outfile, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024*1024):
                    f.write(chunk)
        else:
            print("Missing:", url)


Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/69960403145.csv
Missing: https://www.ncei.noaa.gov/data/global-hourly/access/2018/69960403145.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70026027502.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70063727406.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70133026616.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70200026617.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70219026615.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70261026411.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70272526491.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70296026410.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/access/2018/70316025624.csv
Downloading: https://www.ncei.noaa.gov/data/global-hourly/

In [139]:
import pandas as pd
import numpy as np
import glob
import os

noaa_folder = "noaa_data"  # change if yours is different

noaa_files = glob.glob(os.path.join(noaa_folder, "*.csv"))
len(noaa_files)


2848

In [140]:
def parse_noaa_file(path):
    # Read only the columns we care about to save memory
    usecols = [
        "STATION", "DATE", "TMP", "DEW", "WND", "VIS", "CIG", "SLP"
    ]
    df = pd.read_csv(path, usecols=usecols)
    
    # Convert DATE to datetime and floor to hour
    df["datetime_hour"] = pd.to_datetime(df["DATE"]).dt.floor("h")
    
    # --- Temperature (°C) ---
    tmp = df["TMP"].str.split(",", expand=True)
    df["temp_c"] = pd.to_numeric(tmp[0], errors="coerce") / 10.0
    
    # --- Dewpoint (°C) ---
    dew = df["DEW"].str.split(",", expand=True)
    df["dewpoint_c"] = pd.to_numeric(dew[0], errors="coerce") / 10.0
    
    # --- Visibility (meters) ---
    vis = df["VIS"].str.split(",", expand=True)
    df["visibility_m"] = pd.to_numeric(vis[0], errors="coerce")
    
    # --- Sea-level pressure (hPa) ---
    slp = df["SLP"].str.split(",", expand=True)
    df["slp_hpa"] = pd.to_numeric(slp[0], errors="coerce") / 10.0
    
    # --- Ceiling (meters) ---
    cig = df["CIG"].str.split(",", expand=True)
    df["ceiling_m"] = pd.to_numeric(cig[0], errors="coerce")
    
    # --- Wind: direction (deg) + speed (m/s) ---
    wnd = df["WND"].str.split(",", expand=True)
    # wnd[0] = direction (deg *or* 999 for missing)
    # wnd[2] = speed (m/s * 10)
    df["wind_dir_deg"] = pd.to_numeric(wnd[0], errors="coerce")
    df["wind_speed_mps"] = pd.to_numeric(wnd[3], errors="coerce") / 10.0
    
    # Keep a small tidy table
    df_clean = df[[
        "STATION", "datetime_hour",
        "temp_c", "dewpoint_c",
        "visibility_m", "slp_hpa",
        "ceiling_m", "wind_dir_deg", "wind_speed_mps"
    ]]
    
    return df_clean

weather_frames = []
for f in noaa_files:
    print("Parsing", f)
    weather_frames.append(parse_noaa_file(f))

weather = pd.concat(weather_frames, ignore_index=True)

weather = weather.drop_duplicates(subset=["STATION", "datetime_hour"])
weather.head()


Parsing noaa_data/72278623104_2019.csv
Parsing noaa_data/72250612959_2025.csv
Parsing noaa_data/72477693075_2023.csv
Parsing noaa_data/72607014606_2021.csv
Parsing noaa_data/72250612959_2019.csv
Parsing noaa_data/72278623104_2025.csv
Parsing noaa_data/72429093815_2022.csv
Parsing noaa_data/72317013723_2023.csv
Parsing noaa_data/72421093814_2023.csv
Parsing noaa_data/72428513812_2018.csv
Parsing noaa_data/72261022010_2024.csv
Parsing noaa_data/72467523063_2019.csv
Parsing noaa_data/72506694724_2025.csv
Parsing noaa_data/72462023061_2020.csv
Parsing noaa_data/72261022010_2018.csv
Parsing noaa_data/72428513812_2024.csv
Parsing noaa_data/72433813802_2022.csv
Parsing noaa_data/72467523063_2025.csv
Parsing noaa_data/72506694724_2019.csv
Parsing noaa_data/72403093738_2021.csv
Parsing noaa_data/72747014918_2021.csv
Parsing noaa_data/72793024233_2022.csv
Parsing noaa_data/72268893034_2022.csv
Parsing noaa_data/91212041415_2020.csv
Parsing noaa_data/72480023157_2020.csv
Parsing noaa_data/7263791

,STATION,datetime_hour,temp_c,dewpoint_c,visibility_m,slp_hpa,ceiling_m,wind_dir_deg,wind_speed_mps
0,72278623104,2019-01-01 00:00:00,6.0,5.0,6437,9999.9,1097,280,3.6
1,72278623104,2019-01-01 01:00:00,6.0,4.0,6437,9999.9,579,280,3.1
2,72278623104,2019-01-01 02:00:00,5.0,4.0,6437,9999.9,488,320,4.1
3,72278623104,2019-01-01 03:00:00,5.0,4.0,6437,9999.9,366,999,2.1
4,72278623104,2019-01-01 04:00:00,5.0,4.0,6437,9999.9,122,240,3.6


In [141]:
# Example: FlightDate (YYYY-MM-DD), CRSDepTime (e.g. 945, 1330)
normal_flights["FlightDate"] = pd.to_datetime(normal_flights["FlightDate"])

# Make sure dep time is zero-padded 4-digit string, then split to h/m
t_str = normal_flights["CRSDepTime"].astype(int).astype(str).str.zfill(4)
dep_hour = t_str.str[:2].astype(int)
dep_min = t_str.str[2:].astype(int)

normal_flights["dep_datetime"] = (
    normal_flights["FlightDate"] +
    pd.to_timedelta(dep_hour, unit="h") +
    pd.to_timedelta(dep_min, unit="m")
)

normal_flights["dep_hour"] = normal_flights["dep_datetime"].dt.floor("h")


# Make sure Arr time is zero-padded 4-digit string, then split to h/m
t_str = normal_flights["CRSArrTime"].astype(int).astype(str).str.zfill(4)
Arr_hour = t_str.str[:2].astype(int)
Arr_min = t_str.str[2:].astype(int)

normal_flights["Arr_datetime"] = (
    normal_flights["FlightDate"] +
    pd.to_timedelta(Arr_hour, unit="h") +
    pd.to_timedelta(Arr_min, unit="m")
)

normal_flights["Arr_hour"] = normal_flights["Arr_datetime"].dt.floor("h")

normal_flights[["dep_hour", "Arr_hour"]]


,dep_hour,Arr_hour
0,2018-01-26 13:00:00,2018-01-26 15:00:00
1,2018-01-20 12:00:00,2018-01-20 15:00:00
2,2018-01-27 09:00:00,2018-01-27 11:00:00
3,2018-01-14 16:00:00,2018-01-14 17:00:00
4,2018-01-10 17:00:00,2018-01-10 19:00:00
...,...,...
2268955,2025-01-30 20:00:00,2025-01-30 21:00:00
2268956,2025-01-27 15:00:00,2025-01-27 17:00:00
2268957,2025-01-30 18:00:00,2025-01-30 22:00:00
2268958,2025-01-04 14:00:00,2025-01-04 19:00:00


In [142]:
import pandas as pd

def add_weather_nearest(flights, weather, station_col, time_col, prefix, tol_hours=5):
    flights = flights.copy()
    weather = weather.copy()

    flights[station_col] = flights[station_col].astype(str)
    weather["STATION"] = weather["STATION"].astype(str)
    flights[time_col] = pd.to_datetime(flights[time_col])
    weather["datetime_hour"] = pd.to_datetime(weather["datetime_hour"])

    flights = flights.dropna(subset=[station_col, time_col])
    weather = weather.dropna(subset=["STATION", "datetime_hour"])

    tol = pd.Timedelta(f"{tol_hours}h")
    chunks = []
    wcols = [c for c in weather.columns if c not in ["STATION", "datetime_hour"]]

    for st, fl_st in flights.groupby(station_col, sort=False):
        wx_st = weather[weather["STATION"] == st]
        if wx_st.empty:
            chunks.append(fl_st)
            continue
        fl_st = fl_st.sort_values(time_col).reset_index(drop=True)
        wx_st = wx_st.sort_values("datetime_hour").reset_index(drop=True)
        m = pd.merge_asof(
            fl_st, wx_st,
            left_on=time_col, right_on="datetime_hour",
            direction="nearest", tolerance=tol
        )
        chunks.append(m)

    out = pd.concat(chunks, ignore_index=True)
    out = out.rename(columns={c: f"{prefix}{c}" for c in wcols})
    return out

# 1) origin weather
merged_origin = add_weather_nearest(
    normal_flights, weather,
    station_col="ORIGIN_STATION",
    time_col="dep_datetime",
    prefix="origin_",
    tol_hours=4
)

# 2) destination weather
final_merged = add_weather_nearest(
    merged_origin, weather,
    station_col="DEST_STATION",
    time_col="Arr_datetime",
    prefix="dest_",
    tol_hours=4
)

final_merged

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109,ORIGIN_STATION,DEST_STATION,dep_datetime,dep_hour,Arr_datetime,Arr_hour,STATION_x,datetime_hour_x,origin_temp_c,origin_dewpoint_c,origin_visibility_m,origin_slp_hpa,origin_ceiling_m,origin_wind_dir_deg,origin_wind_speed_mps,STATION_y,datetime_hour_y,dest_temp_c,dest_dewpoint_c,dest_visibility_m,dest_slp_hpa,dest_ceiling_m,dest_wind_dir_deg,dest_wind_speed_mps,STATION,datetime_hour
0,2018,1,1,1,1,2018-01-01,AA,19805,AA,N980UY,1983.0,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,45,37.0,-8.0,0.0,0.0,-1.0,0001-0559,14.0,51.0,635.0,15.0,702,650.0,-12.0,0.0,0.0,-1.0,0700-0759,0.0,NaN,0.0,257.0,253.0,224.0,1.0,2075.0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72278023183,72408013739,2018-01-01 00:45:00,2018-01-01 00:00:00,2018-01-01 07:02:00,2018-01-01 07:00:00,72278023183,2018-01-01 01:00:00,16.1,-2.8,16093.0,1017.3,6706.0,260.0,1.5,72408013739,2018-01-01 07:00:00,-12.2,-21.1,16093.0,1030.2,22000.0,310.0,3.6,NaN,NaT
1,2018,1,1,1,1,2018-01-01,AA,19805,AA,N953UW,1731.0,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,600,557.0,-3.0,0.0,0.0,-1.0,0600-0659,11.0,608.0,734.0,3.0,804,737.0,-27.0,0.0,0.0,-2.0,0800-0859,0.0,NaN,0.0,124.0,100.0,86.0,1.0,666.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72219013874,72408013739,2018-01-01 06:00:00,2018-01-01 06:00:00,2018-01-01 08:04:00,2018-01-01 08:00:00,72219013874,2018-01-01 06:00:00,-3.3,-11.7,16000.0,1026.9,99999.0,340.0,6.7,72408013739,2018-01-01 08:00:00,-12.8,-21.1,16093.0,1030.1,22000.0,310.0,3.1,NaN,NaT
2,2018,1,1,1,1,2018-01-01,OH,20397,OH,N216PS,5563.0,10874,1087402,30647,CAK,"Akron, OH",OH,39,Ohio,44,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,830,818.0,-12.0,0.0,0.0,-1.0,0800-0859,39.0,857.0,955.0,5.0,959,1000.0,1.0,1.0,0.0,0.0,0900-0959,0.0,NaN,0.0,89.0,102.0,58.0,1.0,335.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72521014895,72408013739,

In [147]:
final_merged[(final_merged['STATION'].isna() == True) & (final_merged['STATION_x'].isna() == True)]

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109,ORIGIN_STATION,DEST_STATION,dep_datetime,dep_hour,Arr_datetime,Arr_hour,STATION_x,datetime_hour_x,origin_temp_c,origin_dewpoint_c,origin_visibility_m,origin_slp_hpa,origin_ceiling_m,origin_wind_dir_deg,origin_wind_speed_mps,STATION_y,datetime_hour_y,dest_temp_c,dest_dewpoint_c,dest_visibility_m,dest_slp_hpa,dest_ceiling_m,dest_wind_dir_deg,dest_wind_speed_mps,STATION,datetime_hour
15,2018,1,1,1,1,2018-01-01,AA,19805,AA,N171US,431.0,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida,33,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,1430,1441.0,11.0,11.0,0.0,0.0,1400-1459,30.0,1511.0,1729.0,6.0,1715,1735.0,20.0,20.0,1.0,1.0,1700-1759,0.0,NaN,0.0,165.0,174.0,138.0,1.0,992.0,4,5.0,0.0,9.0,0.0,6.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72202512849,72408013739,2018-01-01 14:30:00,2018-01-01 14:00:00,2018-01-01 17:15:00,2018-01-01 17:00:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72408013739,2018-01-01 17:00:00,-7.2,-21.1,16093.0,1030.1,22000.0,320.0,7.2,NaN,NaT
44,2018,1,1,3,3,2018-01-03,WN,19393,WN,N7876A,1838.0,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida,33,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,945,946.0,1.0,1.0,0.0,0.0,0900-0959,31.0,1017.0,1222.0,4.0,1225,1226.0,1.0,1.0,0.0,0.0,1200-1259,0.0,NaN,0.0,160.0,160.0,125.0,1.0,992.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72202512849,72408013739,2018-01-03 09:45:00,2018-01-03 09:00:00,2018-01-03 12:25:00,2018-01-03 12:00:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72408013739,2018-01-03 12:00:00,-8.9,-18.3,16000.0,1028.3,22000.0,999.0,1.5,NaN,NaT
45,2018,1,1,3,3,2018-01-03,B6,20409,B6,N517JB,376.0,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida,33,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,1021,1139.0,78.0,78.0,1.0,5.0,1000-1059,18.0,1157.0,1408.0,5.0,1303,1413.0,70.0,70.0,1.0,4.0,1300-1359,0.0,NaN,0.0,162.0,154.0,131.0,1.0,992.0,4,70.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72202512849,72408013739,2018-01-03 10:21:00,2018-01-03 10:

In [148]:
cols_to_drop = [
    "STATION_x", "datetime_hour_x",
    "STATION_y", "datetime_hour_y",
    "STATION", "datetime_hour"
]

final_merged = final_merged.drop(columns=[c for c in cols_to_drop if c in final_merged.columns])
final_merged

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109,ORIGIN_STATION,DEST_STATION,dep_datetime,dep_hour,Arr_datetime,Arr_hour,origin_temp_c,origin_dewpoint_c,origin_visibility_m,origin_slp_hpa,origin_ceiling_m,origin_wind_dir_deg,origin_wind_speed_mps,dest_temp_c,dest_dewpoint_c,dest_visibility_m,dest_slp_hpa,dest_ceiling_m,dest_wind_dir_deg,dest_wind_speed_mps
0,2018,1,1,1,1,2018-01-01,AA,19805,AA,N980UY,1983.0,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,45,37.0,-8.0,0.0,0.0,-1.0,0001-0559,14.0,51.0,635.0,15.0,702,650.0,-12.0,0.0,0.0,-1.0,0700-0759,0.0,NaN,0.0,257.0,253.0,224.0,1.0,2075.0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72278023183,72408013739,2018-01-01 00:45:00,2018-01-01 00:00:00,2018-01-01 07:02:00,2018-01-01 07:00:00,16.1,-2.8,16093.0,1017.3,6706.0,260.0,1.5,-12.2,-21.1,16093.0,1030.2,22000.0,310.0,3.6
1,2018,1,1,1,1,2018-01-01,AA,19805,AA,N953UW,1731.0,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,600,557.0,-3.0,0.0,0.0,-1.0,0600-0659,11.0,608.0,734.0,3.0,804,737.0,-27.0,0.0,0.0,-2.0,0800-0859,0.0,NaN,0.0,124.0,100.0,86.0,1.0,666.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72219013874,72408013739,2018-01-01 06:00:00,2018-01-01 06:00:00,2018-01-01 08:04:00,2018-01-01 08:00:00,-3.3,-11.7,16000.0,1026.9,99999.0,340.0,6.7,-12.8,-21.1,16093.0,1030.1,22000.0,310.0,3.1
2,2018,1,1,1,1,2018-01-01,OH,20397,OH,N216PS,5563.0,10874,1087402,30647,CAK,"Akron, OH",OH,39,Ohio,44,14100,1410005,34100,PHL,"Philadelphia, PA",PA,42,Pennsylvania,23,830,818.0,-12.0,0.0,0.0,-1.0,0800-0859,39.0,857.0,955.0,5.0,959,1000.0,1.0,1.0,0.0,0.0,0900-0959,0.0,NaN,0.0,89.0,102.0,58.0,1.0,335.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72521014895,72408013739,2018-01-01 08:30:00,2018-01-01 08:00:00,2018-01-01 09:59:00,2018-01-01 09:00:00,-17.2,-20.6,16093.0,1033.1,22000.0,270.0,3.1,-12.8,-21.1,16093.0,1030.7,22000.0,290.0,3.6
3,2018,1,1,1,1,2018-01-01,YX,20452,YX,N112HQ,473

In [150]:
keep = [

    # ---------- TIME CONTEXT ----------
    'FlightDate',
    'dep_datetime',
    'Arr_datetime',
    'Month',
    'DayOfWeek',
    'DepTimeBlk',

    # ---------- AIRLINE & ROUTE INFO ----------
    'Reporting_Airline',
    'Flight_Number_Reporting_Airline',
    'Origin',
    'Dest',
    'Distance',

    # ---------- DEPARTURE INFO ----------
    'CRSDepTime',
    'DepTime',
    'DepDelay',
    'DepDelayMinutes',
    'DepDel15',
    'DepartureDelayGroups',

    # ---------- ORIGIN WEATHER ----------
    'origin_temp_c',
    'origin_dewpoint_c',
    'origin_visibility_m',
    'origin_slp_hpa',
    'origin_ceiling_m',
    'origin_wind_dir_deg',
    'origin_wind_speed_mps',

    # ---------- ARRIVAL INFO ----------
    'CRSArrTime',
    'ArrTime',
    'ArrDelay',
    'ArrDelayMinutes',
    'ArrDel15',
    'ArrivalDelayGroups',

    # ---------- DESTINATION WEATHER ----------
    'dest_temp_c',
    'dest_dewpoint_c',
    'dest_visibility_m',
    'dest_slp_hpa',
    'dest_ceiling_m',
    'dest_wind_dir_deg',
    'dest_wind_speed_mps',

    # ---------- DELAY BREAKDOWN (POST-OUTCOME) ----------
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay',
]

final = final_merged[keep].copy()
final

,FlightDate,dep_datetime,Arr_datetime,Month,DayOfWeek,DepTimeBlk,Reporting_Airline,Flight_Number_Reporting_Airline,Origin,Dest,Distance,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,origin_temp_c,origin_dewpoint_c,origin_visibility_m,origin_slp_hpa,origin_ceiling_m,origin_wind_dir_deg,origin_wind_speed_mps,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,dest_temp_c,dest_dewpoint_c,dest_visibility_m,dest_slp_hpa,dest_ceiling_m,dest_wind_dir_deg,dest_wind_speed_mps,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2018-01-01,2018-01-01 00:45:00,2018-01-01 07:02:00,1,1,0001-0559,AA,1983.0,PHX,PHL,2075.0,45,37.0,-8.0,0.0,0.0,-1.0,16.1,-2.8,16093.0,1017.3,6706.0,260.0,1.5,702,650.0,-12.0,0.0,0.0,-1.0,-12.2,-21.1,16093.0,1030.2,22000.0,310.0,3.6,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,2018-01-01 06:00:00,2018-01-01 08:04:00,1,1,0600-0659,AA,1731.0,ATL,PHL,666.0,600,557.0,-3.0,0.0,0.0,-1.0,-3.3,-11.7,16000.0,1026.9,99999.0,340.0,6.7,804,737.0,-27.0,0.0,0.0,-2.0,-12.8,-21.1,16093.0,1030.1,22000.0,310.0,3.1,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,2018-01-01 08:30:00,2018-01-01 09:59:00,1,1,0800-0859,OH,5563.0,CAK,PHL,335.0,830,818.0,-12.0,0.0,0.0,-1.0,-17.2,-20.6,16093.0,1033.1,22000.0,270.0,3.1,959,1000.0,1.0,1.0,0.0,0.0,-12.8,-21.1,16093.0,1030.7,22000.0,290.0,3.6,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,2018-01-01 07:00:00,2018-01-01 10:17:00,1,1,0700-0759,YX,4733.0,STL,PHL,814.0,700,720.0,20.0,20.0,1.0,1.0,-18.3,-22.2,16093.0,1045.2,22000.0,320.0,5.7,1017,1031.0,14.0,14.0,0.0,0.0,-12.8,-21.1,16093.0,1030.7,22000.0,290.0,3.6,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,2018-01-01 08:48:00,2018-01-01 10:26:00,1,1,0800-0859,B6,459.0,BOS,PHL,280.0,848,846.0,-2.0,0.0,0.0,-1.0,-17.2,-24.4,16000.0,1022.7,99999.0,290.0,8.2,1026,1007.0,-19.0,0.0,0.0,-2.0,-12.8,-21.1,16093.0,1030.7,22000.0,290.0,3.6,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268955,2024-10-17,2024-10-17 14:43:00,2024-10-17 15:09:00,10,4,1400-1459,AS,64.0,PSG,WRG,31.0,1443,1428.0,-15.0,0.0,0.0,-1.0,6.7,5.0,16093.0,1049.7,1006.0,999.0,1.5,1509,1458.0,-11.0,0.0,0.0,-1.0,8.3,5.7,999999.0,1008.8,99999.0,180.0,4.1,NaN,NaN,NaN,NaN,NaN
2268956,2024-10-21,2024-10-21 09:43:00,2024-10-21 10:27:00,10,1,0900-0959,AS,65.0,KTN,WRG,82.0,943,924.0,-19.0,0.0,0.0,-2.0,1.7,-0.6,16093.0,1019.2,22000.0,310.0,3.1,1027,954.0,-33.0,0.0,0.0,-2.0,2.6,-6.4,999999.0,1018.2,99999.0,90.0,3.1,NaN,NaN,NaN,NaN,NaN
2268957,2024-10-23,2024-10-23 09:43:00,2024-10-23 10:27:00,10,3,0900-0959,AS,65.0,KTN,WRG,82.0,943,937.0,-6.0,0.0,0.0,-1.0,7.2,0.0,16093.0,1020.1,792.0,130.0,5.1,1027,1017.0,-10.0,0.0,0.0,-1.0,5.4,4.5,999999.0,1016.7,99999.0,100.0,1.0,NaN,NaN,NaN,NaN,NaN
2268958,2024-11-08,2024-11-08 14:58:00,2024-11-08 15:25:00,11,5,1400-1459,AS,64.0,PSG,WRG,31.0,1458,1436.0,-22.0,0.0,0.0,-2.0,3.0,2.0,16093.0,9999.9,1128.0,999.0,0.0,1525,1456.0,-29.0,0.0,0.0,-2.0,5.6,4.4,999999.0,1005.8,99999.0,100.0,1.0,NaN,NaN,NaN,NaN,NaN


In [151]:
final.to_csv('finalairline_weather.csv', index=False) 